## 模型性能测试

### LightGBM

In [3]:
import pandas as pd
from joblib import load
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# 加载测试数据集
test_data = pd.read_csv('xzq_p2_vectors.csv')
X_test = test_data.iloc[:, :-1]  # 排除最后一列(label)

# 使用joblib加载已训练的模型
model = load('models/lgb_model.pkl')

# 使用模型进行预测
y_pred = model.predict(X_test)

# 对y_true进行编码转换
labels, uniques = pd.factorize(test_data['label'])
y_true = labels

# 假设这是factorize之后与原标签的对应关系
labels_to_names = {0: 'lijunjie', 1: 'xuzhaoqi', 2: 'wusiyuan', 3: 'chence'}

# 将y_true的整数标签转换为字符串标签
y_true_named = np.vectorize(labels_to_names.get)(y_true)

# 现在y_true_named和y_pred都是字符串形式的标签，可以计算性能指标
precision = precision_score(y_true_named, y_pred, labels=list(labels_to_names.values()), average='weighted')
recall = recall_score(y_true_named, y_pred, labels=list(labels_to_names.values()), average='weighted')
f1 = f1_score(y_true_named, y_pred, labels=list(labels_to_names.values()), average='weighted')

print(f"精确度: {precision:.4f}")
print(f"召回率: {recall:.4f}")
print(f"F1 分数: {f1:.4f}")

精确度: 1.0000
召回率: 1.0000
F1 分数: 1.0000


C:\ANACONDA\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ANACONDA\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ANACONDA\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
